# IMDB: TEXT Classification + BERT + Ax

## Librairies

- Need ``datasets==1.7.0``
- Need ``ax-platform==0.1.20``

Install them from command line if necessary.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath("../../.."))

In [2]:
import io
import re
import pickle
from timeit import default_timer as timer

from tqdm.notebook import tqdm

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer
from transformers import BertModel
from transformers.data.data_collator import DataCollatorWithPadding

from ax import optimize
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting

import esntorch.core.reservoir as res
import esntorch.core.learning_algo as la
import esntorch.core.merging_strategy as ms
import esntorch.core.esn as esn

In [3]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
SEED = 42

## Global variables

In [6]:
pooling_strategy = 'last' # 'mean'

In [7]:
RESULTS_PATH = '/raid/home/jeremiec/Ax_results/ESN_v2' # path of your result folder
CACHE_DIR = '/raid/home/jeremiec/huggingface_datasets' # path of your folder
PARAMS_FILE = 'imdb_params_uniform_'+pooling_strategy+'-pooling.pkl'        # try uniform distribution
RESULTS_FILE = 'imdb_results_esn_uniform_'+pooling_strategy+'-pooling.pkl'  # try uniform distribution

## Dataset

In [8]:
# rename correct column as 'labels': depends on the dataset you load

def tokenize(sample):
    """Tokenize sample"""
    
    sample = tokenizer(sample['text'], truncation=True, padding=False, return_length=True)
    
    return sample
    
def load_and_enrich_dataset(dataset_name, split, cache_dir):
    """
    Load dataset from the datasets library of HuggingFace.
    Tokenize and add length.
    """
    
    # Load dataset
    dataset = load_dataset(dataset_name, split=split, cache_dir=CACHE_DIR)
    
    # Rename label column for tokenization purposes
    dataset = dataset.rename_column('label', 'labels')
    
    # Tokenize data
    dataset = dataset.map(tokenize, batched=True)
    dataset = dataset.rename_column('length', 'lengths')
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels', 'lengths'])
    
    return dataset

In [9]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# train set
full_train_dataset = load_and_enrich_dataset('imdb', split='train', cache_dir=CACHE_DIR).sort("lengths")

# mini train and val sets: 70%/30% of 30%
train_val_datasets = full_train_dataset.train_test_split(train_size=0.3, shuffle=True)
train_val_datasets = train_val_datasets['train'].train_test_split(train_size=0.7, shuffle=True)
train_dataset = train_val_datasets['train'].sort("lengths")
val_dataset = train_val_datasets['test'].sort("lengths")

# test set
test_dataset = load_and_enrich_dataset('imdb', split='test', cache_dir=CACHE_DIR).sort("lengths")

dataset_d = {
    'full_train': full_train_dataset,
    'train': train_dataset,
    'val': val_dataset,
    'test': test_dataset
    }

# dataloaders
dataloader_d = {}
for k, v in dataset_d.items():
    dataloader_d[k] = torch.utils.data.DataLoader(v, batch_size=256, collate_fn=DataCollatorWithPadding(tokenizer))

Reusing dataset imdb (/raid/home/jeremiec/huggingface_datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Loading cached processed dataset at /raid/home/jeremiec/huggingface_datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-b8af816c5d5838c5.arrow
Loading cached sorted indices for dataset at /raid/home/jeremiec/huggingface_datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-94d66c41a45f83b5.arrow
Loading cached split indices for dataset at /raid/home/jeremiec/huggingface_datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-db5cc4fdb7517759.arrow and /raid/home/jeremiec/huggingface_datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-126d6435659c0163.arrow
Reusing dataset imdb (/raid/home/jeremiec/huggingface_datasets/imdb/plain_text/1.0.0/2fdd8b9

In [10]:
dataset_d

{'full_train': Dataset({
     features: ['attention_mask', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 25000
 }),
 'train': Dataset({
     features: ['attention_mask', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 5250
 }),
 'val': Dataset({
     features: ['attention_mask', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 2250
 }),
 'test': Dataset({
     features: ['attention_mask', 'input_ids', 'labels', 'lengths', 'text', 'token_type_ids'],
     num_rows: 25000
 })}

## Optimization

- Parameters

In [11]:
esn_params = {
            'embedding_weights': 'bert-base-uncased', # TEXT.vocab.vectors,
            'distribution' : 'uniform',              # uniform, gaussian
            'input_dim' : 768,                        # dim of encoding!
            'reservoir_dim' : None,
            'bias_scaling' : 0.0,
            'sparsity' : None,
            'spectral_radius' : None,
            'leaking_rate': None,
            'activation_function' : None,
            'input_scaling' : None,
            'mean' : 0.0,
            'std' : 1.0,
            'learning_algo' : None,
            'criterion' : None,
            'optimizer' : None,
            'merging_strategy' : pooling_strategy,
            'lexicon' : None,
            'bidirectional' : False, # False
            'mode' : None,
            'device' : device,
            'seed' : None
             }

In [12]:
def plug_parameters(args_d):
    
    params_d = esn_params.copy()
    
    for k, v in args_d.items():
        if k in params_d.keys():
            params_d[k] = v
            
    return params_d

In [13]:
# args_d = {'reservoir_dim': 1000, 
#      'spectral_radius': 0.9, 
#      'leaking_rate': 0.5, 
#      'input_scaling': 0.1, 
#      'sparsity': 0.99, 
#      'activation_function': 'tanh', 
#      'alpha': 1.0}

# d = plug_parameters(args_d)

# d

- Fitness function

In [14]:
def warm_up(ESN, dataset_d):
    """Warm up the ESN."""
    
    nb_sentences = 10
    
    for i in range(nb_sentences): 

        sentence = dataset_d["train"].select([i])
        dataloader_tmp = torch.utils.data.DataLoader(sentence, 
                                                     batch_size=1, 
                                                     collate_fn=DataCollatorWithPadding(tokenizer))  

        for sentence in dataloader_tmp:
            
            ESN.warm_up(sentence)

In [15]:
def fitness(reservoir_dim=1000,
            spectral_radius=0.9,
            leaking_rate=0.5,
            input_scaling=0.1, 
            sparsity=0.99,
            activation_function='tanh',
            alpha=1.0,
            dataset_d=dataset_d, 
            dataloader_d=dataloader_d, 
            seed_l=[1234321, 420, 888, 1979, 7], # 5 seeds
            return_test_acc=False):
    
    # create esn parameters
    args_d = locals()
    esn_params = plug_parameters(args_d)
        
    # BO mode: param optimization (esn_params['mode'] = 'esn')
    if not return_test_acc:
    
        for seed in tqdm(seed_l):
            
            acc_l = []
            
            # model
            esn_params['mode'] = 'esn'
            esn_params['seed'] = seed
            print('params:', esn_params)
            ESN = esn.EchoStateNetwork(**esn_params)
            ESN.learning_algo = la.RidgeRegression(alpha = alpha)
            ESN = ESN.to(device)

            # warm up
            warm_up(ESN, dataset_d)
            
            # train
            LOSS = ESN.fit(dataloader_d["train"])                           # mini train set
            acc = ESN.predict(dataloader_d["val"], verbose=False)[1].item() # mini val set
            acc_l.append(acc)
            
            # clean objects
            del ESN.learning_algo
            del ESN.criterion
            del ESN.merging_strategy
            del ESN
            torch.cuda.empty_cache()
            
        # results
        return np.mean(acc_l) 
          
    # prediction mode: once params have been optimized
    elif return_test_acc:
        
        # esn
        acc_esn_l = []
        time_esn_l = []
        # custom baseline
        acc_bs_l = []
        time_bs_l = []
        
        for seed in tqdm(seed_l):
            
            for mode in tqdm(['esn', 'linear_layer']):
            
                # model
                esn_params['mode'] = mode
                esn_params['seed'] = seed
                print('params:', esn_params)
                ESN = esn.EchoStateNetwork(**esn_params)
                ESN.learning_algo = la.RidgeRegression(alpha = alpha)
                ESN = ESN.to(device)

                # warm up
                warm_up(ESN, dataset_d)

                # train
                t0 = timer()
                LOSS = ESN.fit(dataloader_d["full_train"])                       # full train set
                t1 = timer()
                acc = ESN.predict(dataloader_d["test"], verbose=False)[1].item() # full test set
                
                # clean objects
                del ESN.learning_algo
                del ESN.criterion
                del ESN.merging_strategy
                del ESN
                torch.cuda.empty_cache()
                
                # results
                if mode == 'esn':
                    time_esn_l.append(t1 - t0)
                    acc_esn_l.append(acc)
                elif mode == 'linear_layer':
                    time_bs_l.append(t1 - t0)
                    acc_bs_l.append(acc)
            
        results_d = {}
        results_d['esn'] = [np.mean(acc_esn_l), np.std(acc_esn_l), np.mean(time_esn_l), np.std(time_esn_l)]
        results_d['linear_layer'] = [np.mean(acc_bs_l), np.std(acc_bs_l), np.mean(time_bs_l), np.std(time_bs_l)]
            
        return results_d

In [16]:
# %%time

# fitness(
#         reservoir_dim=1000,
#         spectral_radius=0.9,
#         leaking_rate=0.5,
#         input_scaling=0.1, 
#         sparsity=0.99,
#         activation_function='tanh',
#         alpha=1.0,
#         dataset_d=dataset_d, 
#         dataloader_d=dataloader_d,
#         seed_l=[1234321, 7],
#         return_test_acc=True
# )

In [17]:
def wrapped_fitness(d, return_test_acc=False):
    
    return fitness(reservoir_dim=d['reservoir_dim'], # will be in the loop 
                   spectral_radius=d['spectral_radius'],
                   leaking_rate=d['leaking_rate'],
                   input_scaling=d['input_scaling'],
                   sparsity=d['sparsity'],
                   activation_function=d['activation_function'],
                   alpha=d['alpha'],
                   dataset_d=dataset_d,
                   dataloader_d=dataloader_d,
                   return_test_acc=return_test_acc)

In [18]:
# *** WARNING *** DO NO EXECUTE NEXT CELLS IF BIDIRECTIONAL MODE (OPTIM ALREADY DONE)

- Otpimization

In [19]:
best_params_d = {}

for res_dim in tqdm([500, 1000, 3000, 5000]):
    
    best_parameters, best_values, experiment, model = optimize(
            parameters=[
              {
                "name": "reservoir_dim",
                "value_type": "int",
                "type": "fixed",
                "value": res_dim,
              },
              {
                "name": "activation_function",
                "value_type": "str",
                "type": "fixed",
                "value": 'tanh',
              },
              {
                "name": "spectral_radius",
                "value_type": "float",
                "type": "choice",
                "values": [0.5, 1.0, 1.5],
              },
              {
                "name": "leaking_rate",
                "value_type": "float",
                "type": "choice",
                "values": [0.1, 0.5, 0.9],
              },
              {
                "name": "input_scaling",
                "value_type": "float",
                "type": "choice",
                "values": [0.1, 1.0],
              },
              {
                "name": "sparsity",
                "value_type": "float",
                "type": "choice",
                "values": [0.0, 0.9],
              },
              {
                "name": "alpha",
                "value_type": "float",
                "type": "choice",
                "values": [0.1, 1.0, 10.0, 100.0],
              }
            ],
            # Booth function
            evaluation_function = wrapped_fitness,
            minimize = False,
            objective_name = 'val_accuracy',
            total_trials = 20 # nb iterations here
        )
    
    # results
    best_params_d[res_dim] = {}
    best_params_d[res_dim]['best_parameters'] = best_parameters
    best_params_d[res_dim]['best_values'] = best_values
    best_params_d[res_dim]['experiment'] = experiment
    # best_params_d[res_dim]['model'] = model

  0%|          | 0/4 [00:00<?, ?it/s]

[INFO 01-13 19:17:35] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 01-13 19:17:35] ax.service.managed_loop: Started full optimization with 20 steps.
[INFO 01-13 19:17:35] ax.service.managed_loop: Running optimization trial 1...


  0%|          | 0/5 [00:00<?, ?it/s]

params: {'embedding_weights': 'bert-base-uncased', 'distribution': 'uniform', 'input_dim': 768, 'reservoir_dim': 500, 'bias_scaling': 0.0, 'sparsity': 0.9, 'spectral_radius': 1.0, 'leaking_rate': 0.5, 'activation_function': 'tanh', 'input_scaling': 1.0, 'mean': 0.0, 'std': 1.0, 'learning_algo': None, 'criterion': None, 'optimizer': None, 'merging_strategy': 'last', 'lexicon': None, 'bidirectional': False, 'mode': 'esn', 'device': device(type='cuda'), 'seed': 1234321}


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Model downloaded: bert-base-uncased
params: {'embedding_weights': 'bert-base-uncased', 'distribution': 'uniform', 'input_dim': 768, 'reservoir_dim': 5000, 'bias_scaling': 0.0, 'sparsity': 0.0, 'spectral_radius': 1.5, 'leaking_rate': 0.1, 'activation_function': 'tanh', 'input_scaling': 1.0, 'mean': 0.0, 'std': 1.0, 'learning_algo': None, 'criterion': None, 'optimizer': None, 'merging_strategy': 'last', 'lexicon': None, 'bidirectional': False, 'mode': 'esn', 'device': device(type='cuda'), 'seed': 1979}


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased
params: {'embedding_weights': 'bert-base-uncased', 'distribution': 'uniform', 'input_dim': 768, 'reservoir_dim': 5000, 'bias_scaling': 0.0, 'sparsity': 0.0, 'spectral_radius': 1.5, 'leaking_rate': 0.1, 'activation_function': 'tanh', 'input_scaling': 1.0, 'mean': 0.0, 'std': 1.0, 'learning_algo': None, 'criterion': None, 'optimizer': None, 'merging_strategy': 'last', 'lexicon': None, 'bidirectional': False, 'mode': 'esn', 'device': device(type='cuda'), 'seed': 7}


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased


In [20]:
best_params_d

{500: {'best_parameters': {'spectral_radius': 1.0,
   'leaking_rate': 0.1,
   'input_scaling': 0.1,
   'sparsity': 0.9,
   'alpha': 100.0,
   'reservoir_dim': 500,
   'activation_function': 'tanh'},
  'best_values': ({'val_accuracy': 80.53333282470703},
   {'val_accuracy': {'val_accuracy': nan}}),
  'experiment': SimpleExperiment(None)},
 1000: {'best_parameters': {'spectral_radius': 1.0,
   'leaking_rate': 0.1,
   'input_scaling': 0.1,
   'sparsity': 0.0,
   'alpha': 100.0,
   'reservoir_dim': 1000,
   'activation_function': 'tanh'},
  'best_values': ({'val_accuracy': 82.44444274902344},
   {'val_accuracy': {'val_accuracy': nan}}),
  'experiment': SimpleExperiment(None)},
 3000: {'best_parameters': {'spectral_radius': 1.0,
   'leaking_rate': 0.1,
   'input_scaling': 0.1,
   'sparsity': 0.0,
   'alpha': 100.0,
   'reservoir_dim': 3000,
   'activation_function': 'tanh'},
  'best_values': ({'val_accuracy': 83.82221984863281},
   {'val_accuracy': {'val_accuracy': nan}}),
  'experiment': S

In [21]:
# save parameters

with open(os.path.join(RESULTS_PATH, PARAMS_FILE), 'wb') as fh:
    pickle.dump(best_params_d, fh)

In [19]:
# load results
with open(os.path.join(RESULTS_PATH, PARAMS_FILE), 'rb') as fh:
    best_params_d = pickle.load(fh)

In [21]:
# best_params_d

## Simulations with best params

In [ ]:
# results

results_d = {}

for res_dim in tqdm([500, 1000, 3000, 5000]):
    
    best_parameters = best_params_d[res_dim]['best_parameters']
    d = wrapped_fitness(best_parameters, return_test_acc=True)
    results_d[res_dim] = d
    print("Experiment finished.")

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

params: {'embedding_weights': 'bert-base-uncased', 'distribution': 'uniform', 'input_dim': 768, 'reservoir_dim': 500, 'bias_scaling': 0.0, 'sparsity': 0.9, 'spectral_radius': 1.0, 'leaking_rate': 0.1, 'activation_function': 'tanh', 'input_scaling': 0.1, 'mean': 0.0, 'std': 1.0, 'learning_algo': None, 'criterion': None, 'optimizer': None, 'merging_strategy': 'last', 'lexicon': None, 'bidirectional': False, 'mode': 'esn', 'device': device(type='cuda'), 'seed': 1234321}


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased
params: {'embedding_weights': 'bert-base-uncased', 'distribution': 'uniform', 'input_dim': 768, 'reservoir_dim': 500, 'bias_scaling': 0.0, 'sparsity': 0.9, 'spectral_radius': 1.0, 'leaking_rate': 0.1, 'activation_function': 'tanh', 'input_scaling': 0.1, 'mean': 0.0, 'std': 1.0, 'learning_algo': None, 'criterion': None, 'optimizer': None, 'merging_strategy': 'last', 'lexicon': None, 'bidirectional': False, 'mode': 'linear_layer', 'device': device(type='cuda'), 'seed': 1234321}


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model downloaded: bert-base-uncased


In [ ]:
results_d

In [ ]:
# save results

with open(os.path.join(RESULTS_PATH, RESULTS_FILE), 'wb') as fh:
    pickle.dump(results_d, fh)

In [ ]:
# # load results
# with open(os.path.join(RESULTS_PATH, RESULTS_FILE), 'rb') as fh:
#     results_d = pickle.load(fh)

In [ ]:
# results_d